# `__new__`


`object.__new__(cls[, ...])`  
`__new__` is called to create a new instance of class `cls`. It is a static method, which takes the class of which an instances was requested as its first argument. Remaining are arguments passed into the constructor. The return value should be **a** new object instance (if this is not returned, the instance is not created)



Typically call `super().__new(cls[, ...])`. 

`__init__` vs `__new__`  

According to the python docs, `__new__` was for customizing instance creation when subclassing built-int types. Since it's invoked before `__init__`, it is called with the CLASS as it's first argument (whereas `__init__` is called with an instance as its first and doesn't return anything)

`__new__()` is intended mainly to allow subclasses of immutable types (like int, str, or tuple) to customize instance creation. It is also commonly overridden in custom metaclasses in order to customize class creation.

Because `__new__()` and `__init__()` work together in constructing objects (`__new__()` to create it, and `__init__()` to customize it), no non-None value may be returned by `__init__`; doing so will cause a TypeError to be raised at runtime.


In [34]:
# making the call-order of __init__ and __new__ clear
class A:
    def __new__(cls: type,*args, **kwargs):
        print(f'{cls}.__new__')
        print(f'args: {args}')
        print(f'kwargs: {kwargs}')
        # actually creates the object
        return object().__new__(A, **kwargs)

    def __init__(self, *args, **kwargs) -> None:
        # at this point the object is already created
        print(f'{self}.__init__')
        print(f'args: {args}')
        print(f'kwargs: {kwargs}')

a = A()


<class '__main__.A'>.__new__
args: ()
kwargs: {}
<__main__.A object at 0x7f84ecf9fc70>.__init__
args: ()
kwargs: {}


Exploring the execution order without using the `class` keyword 

In [36]:
type(a), type(type(a)), type(type(type(a))) # hmm

(__main__.A, type, type)

If we use the `type` function to create a new class (EXACTLY the same as above), since `class` is syntactic sugar for doing something similar to the following:

In [40]:
# creating classes without using the word class

# set the functions to create class
def __new__(cls: type,*args, **kwargs):
    print(f'{cls}.__new__')
    print(f'args: {args}')
    print(f'kwargs: {kwargs}')
    # actually creates the object
    return object().__new__(A, **kwargs)

def __init__(self, *args, **kwargs) -> None:
    # at this point the object is already created
    print(f'{self}.__init__')
    print(f'args: {args}')
    print(f'kwargs: {kwargs}')

name = 'A'
bases = ()
namespace = {

        '__init__': __init__,
        '__new__': __new__
}

A = type(name, bases, namespace) # THIS is how classes are created
# since every class is an instance of type

# creating an instance
a = A() # same as with the class keyword

<class '__main__.A'>.__new__
args: ()
kwargs: {}
<__main__.A object at 0x7f84ece00ac0>.__init__
args: ()
kwargs: {}


# Implementing the Factory Pattern

the `__new__` function determines what `type` of object to return based on the inputs. This is important, since if it was done in `__init__`, the object would have been created *prior*.  

In [49]:
from typing import TypeVar, Generic, List, Union, overload
from typing_extensions import Protocol
from datetime import datetime
from numpy import datetime64
from pandas import DatetimeIndex
from typing import overload

T = TypeVar("T", covariant=True)
S = TypeVar("S")

class Index:

    @overload
    def __new__(cls, values: List[datetime64]) -> DatetimeIndex: ...
    @overload
    def __new__(cls, values: List[datetime]) -> DatetimeIndex: ...
    @overload
    def __new__(cls, values: List[S]) -> DefaultIndex: ...

    def __new__(cls, values):
        if type(values[0]) in (datetime, datetime64):
            print('DateTime __new__')
            cls = DatetimeIndex
        else:
            print('Default __new__')
            cls = DefaultIndex
        return object.__new__(cls)


class DefaultIndex(Index, Generic[S]):
    def __init__(self, values: List[S]):
        self.values = values

    def first(self):
        return self.values[0]



In [52]:
from numpy import datetime64
from datetime import date

di = DefaultIndex([datetime64('2022-01-01')])

type(di)

DateTime __new__


pandas.core.indexes.datetimes.DatetimeIndex

In [35]:
# order of execution

class BaseClass:
    def __init__(self, *args, **kwargs):
        print('init')
        for k in kwargs.keys():
            self.k = kwargs[k] 

    def __new__(cls, *args, **kwargs):
        print('new')
        super().__init__(BaseClass)

    def __prepare__(metacls, name, bases):
        print('prepare')
        return dict()

In [41]:
class A:
    def __new__(cls, name, bases, namespaces, **kwargs):
        annotations = namespaces.get('__annotations__', {})
        namespaces['__annotations__'] = annotations
        return super().__new__(cls, name, bases, namespaces, **kwargs)

A()

TypeError: __new__() missing 3 required positional arguments: 'name', 'bases', and 'namespaces'

In [16]:
# example from GPT

class Shape:
    def __new__(cls, *args, **kwargs):
        shape_type = kwargs['shape_type']
        print(f'__new__: {cls}, {args}, {kwargs}')
        if shape_type == 'circle':
            return Circle(*args, **kwargs)
        elif shape_type == 'square':
            return Square(*args, **kwargs)
        else:
            raise ValueError('Invalid Shape type')

    def __init__(self, *args, **kwargs):
        pass

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

class Square(Shape):
    def __init__(self, side):
        self.side= side 



# Implementing the Flyweight Pattern

The flyweight pattern
is designed for conserving memory; if we have hundreds of thousands of similar
objects, combining similar properties into a flyweight can have an enormous impact
on memory consumption. It is common for programming solutions that optimize
CPU, memory, or disk space result in more complicated code than their unoptimized
brethren. It is therefore important to weigh up the tradeoffs when deciding between
code maintainability and optimization. When choosing optimization, try to use
patterns such as flyweight to ensure that the complexity introduced by optimization
is confined to a single (well documented) section of the code

In [27]:
import weakref
class CarModel:

    _models = weakref.WeakValueDictionary()

    def __new__(cls, model_name, *args, **kwargs):
        model = cls._models.get(model_name)

        if not model:
            model = super().__new__(cls)
        cls._models[model_name] = model
        return model

    
    def __init__(self, model_name, air=False, tilt=False,
        cruise_control=False, power_locks=False,
        alloy_wheels=False, usb_charger=False):
        if not hasattr(self, "initted"):
            self.model_name = model_name
            self.air = air
            self.tilt = tilt
            self.cruise_control = cruise_control
            self.power_locks = power_locks
            self.alloy_wheels = alloy_wheels
            self.usb_charger = usb_charger
            self.initted=True

In [29]:
c = CarModel('CRV', usb_charger=True)
hasattr(c, 'initted')

True